## 命名空间管理

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
sess = tf.Session()

### 1.使用tf.Variable()和tf.get_variable()创建变量

#### 1.1 使用tf.variable 创建，如名称一样则自动处理

In [2]:
v1 = tf.Variable(initial_value=[1.0], name='v')
v2 = tf.Variable(initial_value=[2.0], name='v')
v3 = tf.Variable(initial_value=[1.0, 2.0], name='v')

In [3]:
print("v1", v1)
print("v2", v2)
print("v3", v3)

v1 <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
v2 <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
v3 <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>


#### 1.2 使用tf.get_variable创建变量，不能用一样的name

In [4]:
gv1 = tf.get_variable(name='gv', shape=[2,3], initializer=tf.truncated_normal_initializer())

非要使用一样name，需要结合tf.variable_scope&reuse

#### 1.3 tf.Varibale()和tf.get_variable 同时创建变量，会自动处理

In [5]:
var1 = tf.Variable(initial_value=[1.0], name='var', trainable=False)
var2 = tf.get_variable(name='var', shape=[2,3])
var3 = tf.Variable(initial_value=[1.0], name='var')
# var4 = tf.get_variable(name='var', shape=[2,3])  # 报错
print('var1', var1)
print('var2', var2)
print('var3', var3)

var1 <tf.Variable 'var:0' shape=(1,) dtype=float32_ref>
var2 <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
var3 <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>


#### 1.4使用tf.placeholder()创建占位符

In [6]:
ph1 = tf.placeholder(dtype=tf.float32, shape=[1,3], name='ph')
ph2 = tf.placeholder(dtype=tf.float32, shape=[2,3], name='ph')

print('ph1:', ph1)
print('ph2:', ph2)

ph1: Tensor("ph:0", shape=(1, 3), dtype=float32)
ph2: Tensor("ph_1:0", shape=(2, 3), dtype=float32)


可以看出，placeholder也可以自动处理

##### 1.5获取全部变量和trainable变量

In [7]:
all_vars = tf.global_variables()
for i in range(len(all_vars)):
    print(i, all_vars[i])

0 <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
1 <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
2 <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>
3 <tf.Variable 'gv:0' shape=(2, 3) dtype=float32_ref>
4 <tf.Variable 'var:0' shape=(1,) dtype=float32_ref>
5 <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
6 <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>


In [8]:
all_trainable_vars = tf.trainable_variables()
for i in range(len(all_trainable_vars)):
    print(i,all_trainable_vars[i])

0 <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
1 <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
2 <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>
3 <tf.Variable 'gv:0' shape=(2, 3) dtype=float32_ref>
4 <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
5 <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>


tf.global_variables()和tf.trainable_variables()用来获取所有变量和所有可训练的变量。
这个后期会用到。比如，定义两个不同的优化器，各自优化部分参数，这个术后需要给他们传入各自需要优化的变量，这时候他们只优化list中的变量

### 2.使用tf.name_scope()和tf.variable_scope()管理命名空间

#### 2.1 使用tf.name_scope()

In [9]:
with tf.name_scope('ns1') as ns:
    ns_v1 = tf.Variable(initial_value=[1.0],name='V')
    ns_gv1 = tf.get_variable(name='V',shape=[2,3])
    ns_v2 = tf.Variable(initial_value=[1.0], name='V')

In [10]:
print('ns_v1', ns_v1)
print('ns_gv1', ns_gv1)
print('ns_v2', ns_v2)

ns_v1 <tf.Variable 'ns1/V:0' shape=(1,) dtype=float32_ref>
ns_gv1 <tf.Variable 'V:0' shape=(2, 3) dtype=float32_ref>
ns_v2 <tf.Variable 'ns1/V_1:0' shape=(1,) dtype=float32_ref>


- tf.Variable创建的变量会自动在name前加上scope name
- tf.get_variable创建的变量，不受name scope影响

#### 2.2使用tf.variable_scope()

In [11]:
with tf.variable_scope('vs1') as vs:
    vs_v1 = tf.Variable(initial_value=[1.0], name='v')
    vs_gv1 = tf.get_variable(name='v', shape=[2,3])
    # vs_gv2 = tf.get_variable(name='v', shape=[1,3])  # ValueError: Variable vs1/v already exists

print('vs_v1', vs_v1)
print('vs_gv1', vs_gv1)

vs_v1 <tf.Variable 'vs1/v:0' shape=(1,) dtype=float32_ref>
vs_gv1 <tf.Variable 'vs1/v_1:0' shape=(2, 3) dtype=float32_ref>


- 使用tf.Variable()创建的变量，会自动的在name前面加入variable scope name
- 使用tf.get_variable创建的变量，也会自动地在name前加上variable scope name

#### 2.3 tf.variable_scope()中设置reuse=True

python中is用来判断是不是指向同一对象，==用来判断内容是否一致

In [12]:
a = [1,2,3]
b = [1,2,3]
c = a
print('a is b:', a is b)
print('a is c:', a is c)
c[0]=555
b[1]=666
print('a',a)
print('b',b)
print('c',c)

a is b: False
a is c: True
a [555, 2, 3]
b [1, 666, 3]
c [555, 2, 3]


tensorflow中，通过变量的name来区分不同对象，可以通过reuse实现变量共享

In [13]:
with tf.variable_scope('vs1') as vs:
    vs_gv2 = tf.get_variable(name='v2', shape=[2,3]) 
    # vs_gv3 = tf.get_variable(name='v', shape=[2,3])  # ValueError: Variable vs1/v already exists  
    
print('vs_gv2', vs_gv2)

vs_gv2 <tf.Variable 'vs1/v2:0' shape=(2, 3) dtype=float32_ref>


In [14]:
with tf.variable_scope('vs1', reuse=True) as vs:
    vs_gv3 = tf.get_variable(name='v', shape=[2,3])  
    
print('vs_gv3', vs_gv3)
print('vs_gv3 is vs_gv1: ', vs_gv3 is vs_gv1)

vs_gv3 <tf.Variable 'vs1/v_1:0' shape=(2, 3) dtype=float32_ref>
vs_gv3 is vs_gv1:  True


当我们在同一个variable_scope下定义一个已经存在的变量name='V'时，需要设置reuse=true,指向同一对象

### 3.总结

一般建议通过tf.variable_scope()和tf.get_variable()一起进行变量管理和权值共享

### 4.示例

下面通过两个例子感受下命名管理

In [15]:
def fc(X, out_size):
    '''
    全连接层
    Args:
        X: 2-D tensor [batch_size, in_size]
        out_size: the size of output tensor
    Returns:
        h_fc: 2-D tensor, [batch_size, out_size]
    '''
    in_size = X.shape[1] # 特征维度
    W = tf.get_variable('weight', shape=[in_size, out_size], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('bias', [out_size], initializer=tf.zeros_initializer())
    h_fc = tf.nn.relu(tf.nn.xw_plus_b(X, W, b), name='relu')
    return h_fc

#### 4.1 正确的例子

In [16]:
batch_size = 128
feature_size = 50
fc1_size = 64
fc2_size = 32
X_input = tf.placeholder(dtype=tf.float32, shape=[batch_size, feature_size], name='X_input')

with tf.variable_scope('fc1') as vs:
    h_fc1 = fc(X_input, out_size=fc1_size)
with tf.variable_scope('fc2') as vs:
    h_fc2 = fc(h_fc1, out_size=fc2_size)

all_vars = tf.global_variables()
for i in range(len(all_vars)):
    print('var {}: '.format(i), all_vars[i])

var 0:  <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
var 1:  <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
var 2:  <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>
var 3:  <tf.Variable 'gv:0' shape=(2, 3) dtype=float32_ref>
var 4:  <tf.Variable 'var:0' shape=(1,) dtype=float32_ref>
var 5:  <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
var 6:  <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>
var 7:  <tf.Variable 'ns1/V:0' shape=(1,) dtype=float32_ref>
var 8:  <tf.Variable 'V:0' shape=(2, 3) dtype=float32_ref>
var 9:  <tf.Variable 'ns1/V_1:0' shape=(1,) dtype=float32_ref>
var 10:  <tf.Variable 'vs1/v:0' shape=(1,) dtype=float32_ref>
var 11:  <tf.Variable 'vs1/v_1:0' shape=(2, 3) dtype=float32_ref>
var 12:  <tf.Variable 'vs1/v2:0' shape=(2, 3) dtype=float32_ref>
var 13:  <tf.Variable 'fc1/weight:0' shape=(50, 64) dtype=float32_ref>
var 14:  <tf.Variable 'fc1/bias:0' shape=(64,) dtype=float32_ref>
var 15:  <tf.Variable 'fc2/weight:0' shape=(64, 32) dtype=float32_ref